In [1]:
import scipy
import scipy.sparse
import numpy
import os
import re
import multiprocessing
import subprocess
import pickle
import matplotlib.pyplot
import time
import statistics
import sklearn


In [2]:
# using full dataset
CSR_mat_full = scipy.sparse.load_npz('./testdata/sparse_matrix_mumbach.npz')
str(CSR_mat_full)
print("total number of reads imported", CSR_mat_full.sum()/2)

total number of reads imported 204245296.0


In [3]:
##############
#thing to make it faster
##############
CSR_mat_full = CSR_mat_full[0:584662,0:584662]
print("MATRIX sliced to get only chromosome 1")
print("total number of reads in chromosome 1", CSR_mat_full.sum()/2)
print("matrix size", CSR_mat_full.shape)

MATRIX sliced to get only chromosome 1
total number of reads in chromosome 1 15886652.0
matrix size (584662, 584662)


In [5]:
scipy.sparse.save_npz('./testdata/sparse_matrix_mumbach_chr1.npz',CSR_mat_full)

In [6]:
#needed to load the variables
with open("./testdata/variables.pi","rb") as picklefile:
    frag_index,frag_prop,frag_amount,valid_chroms,chroms_offsets,distribution_nice_fragments = pickle.load(picklefile)


In [6]:
%matplotlib qt
matplotlib.pyplot.scatter(range(1000),distribution_nice_fragments)

In [14]:
CSR_mat_full.diagonal().sum()

1599232.0

In [7]:
%matplotlib qt
matplotlib.pyplot.plot(CSR_mat_full.diagonal())
matplotlib.pyplot.plot(CSR_mat_full.diagonal(k=1).tolist())
matplotlib.pyplot.plot(CSR_mat_full.diagonal(k=2).tolist())

In [18]:
#preparing data for distribution plots
distribution = [[] for i in range(10000)]
diagonal=CSR_mat_full.diagonal(k=0).tolist()[1:]
lengths=[]
for i in frag_prop:
    lengths.append(i[3])
       
for i in range(len(lengths)):
    try:
        distribution[lengths[i]].append(diagonal[i])
    except:
        continue

results = []        
for i in distribution:
    if i:
        results.append(statistics.mean(i))
    else:
        results.append(1000)
        
for i in range(len(lengths)):
    try:
        if diagonal[i] != 0:
            distribution[lengths[i]].append(diagonal[i])
    except:
        continue        

results2 = []        
for i in distribution:
    if i:
        results2.append(statistics.mean(i))
    else:
        results2.append(1000)     

In [19]:
#plotting avarage reads vs length
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
ax.scatter(range(10000),results)
ax.scatter(range(10000),results2)
fig.show()

In [10]:
%matplotlib qt
#scatter of lenght vs number of reads
fig, ax = matplotlib.pyplot.subplots()
lengths=[]
for i in frag_prop:
    lengths.append(i[3])
ax.scatter(lengths[:584662],CSR_mat_full.diagonal().tolist()[:584662],alpha=0.01)
ax.set_xlim((0, 5000))
ax.set_ylim((0, 5000))
fig.show()